In [3]:
import numpy as np
import cv2

# Define the parameters
dt = 0.1  # Time step
kernel = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])

# Pad the kernel to match the size of the image
padding = (kernel.shape[0] - 1) // 2
kernel_padded = np.pad(kernel, padding, 'constant')

# Define the hyperparameter lambda
lambda_value = 0.1

# Function to compute g(s^2)
def g(s_squared):
    return 1 / (1 + (s_squared + lambda_value**2))

# Function to apply the update rule
def denoise_image(image, kernel):
    k_u = cv2.filter2D(image, -1, kernel)
    phi = k_u*g(k_u**2)
    p = dt * phi 
    return image + p

# Load the image
image = cv2.imread('noisy_cameraman.jpg', 0)
image = image.astype(np.float32) / 255.0

denoised_image = image.copy()
for _ in range(20):
    denoised_image = denoise_image(denoised_image, kernel_padded)

cv2.imwrite('denoised_peronaMalik.jpg', denoised_image * 255.0)

True

In [27]:
import numpy as np
import cv2

# Define the parameters
dt = 0.1  # Time step
kernel = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])

# Pad the kernel to match the size of the image
padding = (kernel.shape[0] - 1) // 2
kernel_padded = np.pad(kernel, padding, 'constant')

# Define the hyperparameter lambda
lambda_value = 0.1

# ReLU activation function
def relu(x):
    return np.maximum(0, x)

# Function to compute phi(x) using ReLU activation
def g(x):
    return relu(x)

# Function to apply the update rule
def denoise_image(image, kernel):
    k_u = cv2.filter2D(image, -1, kernel)
    p = dt * k_u * g(k_u)
    return image + p

# Load the image
image = cv2.imread('noisy_cameraman.jpg', 0)
image = image.astype(np.float32) / 255.0

denoised_image = image.copy()
for _ in range(20):
    denoised_image = denoise_image(denoised_image, kernel_padded)

cv2.imwrite('denoised_ReLU.jpg', denoised_image * 255.0)

True

In [1]:
import numpy as np
import cv2
from math import sqrt

# Define the parameters
dt = 0.1  # Time step
kernel = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])

# Pad the kernel to match the size of the image
padding = (kernel.shape[0] - 1) // 2
kernel_padded = np.pad(kernel, padding, 'constant')

# Define the hyperparameter lambda
lambda_value = 0.1

# Function to compute g(s^2)
def g(s_squared):
    denom = np.sqrt(1 + s_squared/lambda_value**2)
    return np.sqrt(s_squared)/denom

# Function to apply the update rule
def denoise_image(image, kernel):
    k_u = cv2.filter2D(image, -1, kernel)
    phi = k_u*g(k_u**2)
    p = dt * phi
    return image + p

# Load the image
image = cv2.imread('noisy_cameraman.jpg', 0)
image = image.astype(np.float32) / 255.0

denoised_image = image.copy()
for _ in range(20):
    denoised_image = denoise_image(denoised_image, kernel_padded)

cv2.imwrite('denoised_charbonier.jpg', denoised_image * 255.0)

True

### kernel trial

In [3]:


import numpy as np
import cv2

# Define the parameters
dt = 0.1  # Time step
kernel = np.array([[5.09229699e-09,  1.00000000e+00,  5.40467264e-10], [ 1.00000000e+00, -4.00000000e+00,  1.00000000e+00], [-2.81036184e-09,  1.00000000e+00,  1.97387560e-08]])

# Pad the kernel to match the size of the image
padding = (kernel.shape[0] - 1) // 2
kernel_padded = np.pad(kernel, padding, 'constant')

# Define the hyperparameter lambda
lambda_value = 0.1

# Function to compute g(s^2)
def g(s_squared):
    return 1 / (1 + (s_squared + lambda_value**2))

# Function to apply the update rule
def denoise_image(image, kernel):
    k_u = cv2.filter2D(image, -1, kernel)
    phi = k_u*g(k_u**2)
    p = dt * phi 
    return image + p

# Load the image
image = cv2.imread('noisy_cameraman.jpg', 0)
image = image.astype(np.float32) / 255.0

denoised_image = image.copy()
for _ in range(20):
    denoised_image = denoise_image(denoised_image, kernel_padded)

cv2.imwrite('denoised_peronaMalik_test2.jpg', denoised_image * 255.0)

True

In [ ]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm
from scipy.ndimage import convolve
from scipy.optimize import minimize


# Define the parameters
dt = 0.1  # Time step
initial_kernel = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])  # Initial kernel

# Pad the kernel to match the size of the image
padding = (initial_kernel.shape[0] - 1) // 2
initial_kernel_padded = np.pad(initial_kernel, padding, 'constant')

# Define the hyperparameter lambda
lambda_value = 0.1

# Function to compute g(s^2)
def g(s_squared):
    return 1 / (1 + (s_squared + lambda_value**2))

# Function to apply the update rule
def denoise_image(image, kernel):
    k_u = convolve(image, kernel, mode='constant')
    phi = k_u * g(k_u**2)
    p = dt * phi
    return image + p
def objective_function(kernel_params, clean_images, noisy_images):
    kernel = kernel_params.reshape((3, 3))
    kernel_padded = np.pad(kernel, padding, 'constant')
    error = 0
    for clean_image, noisy_image in zip(clean_images, noisy_images):
        denoised_image = denoise_image(noisy_image, kernel_padded)
        error += np.mean((denoised_image - clean_image) ** 2)  # Example error metric (MSE)
    return error


# Load the clean and noisy images from directories
clean_dir = r'D:\Desktop\clean_image'  # Modify with the correct path
noisy_dir = r'D:\Desktop\noisy_image'  # Modify with the correct path
clean_images = []
noisy_images = []
for filename in os.listdir(clean_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        clean_image_path = os.path.join(clean_dir, filename)
        clean_image = Image.open(clean_image_path).convert('L')
        clean_image = np.array(clean_image, dtype=np.float32) / 255.0
        clean_images.append(clean_image)

        noisy_image_path = os.path.join(noisy_dir, filename)
        noisy_image = Image.open(noisy_image_path).convert('L')
        noisy_image = np.array(noisy_image, dtype=np.float32) / 255.0
        noisy_images.append(noisy_image)

# Flatten initial kernel for optimization
initial_params = initial_kernel.flatten()

# Optimize the kernel parameters with tqdm progress bar
with tqdm(total=500, ncols=80, desc='Optimizing Kernel') as pbar:
    def update_progress(_):
        pbar.update(1)

    # Optimize the kernel parameters
    optimized_params = minimize(objective_function, initial_params, args=(clean_images, noisy_images),
                                callback=update_progress, options={'maxiter': 500})

# Reshape optimized parameters to obtain the trained kernel
trained_kernel = optimized_params.x.reshape((3, 3))

# Pad the trained kernel
trained_kernel_padded = np.pad(trained_kernel, padding, 'constant')

# Load the test image
test_image_path = r'D:\Desktop\test_image.jpeg'  # Modify with the correct path
test_image = Image.open(test_image_path).convert('L')
test_image = np.array(test_image, dtype=np.float32) / 255.0

# Denoise the test image using the trained kernel
denoised_image = denoise_image(test_image, trained_kernel_padded)

# Save the denoised image
denoised_image = (denoised_image * 255.0).astype(np.uint8)
denoised_image_path = r'D:\Desktop\denoised_image.jpg'  # Modify with the correct path
Image.fromarray(denoised_image).save(denoised_image_path)

# Compute PSNR
mse = np.mean((test_image - denoised_image / 255.0) ** 2)
psnr = 20 * np.log10(1.0 / np.sqrt(mse))

# Print the results
print("Best Kernel:")
print(trained_kernel)
print("PSNR of Test Image:")
print(psnr)